# Clifford+RZ Weak Sampling (With Qrack)

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
width = 36
depth = 18
max_magic = 9

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

import os
import collections
import math
import random

from pyqrack import QrackSimulator, Pauli
from mitiq import zne
from qiskit.circuit.quantumcircuit import QuantumCircuit
from mitiq.zne.scaling.folding import fold_global
from mitiq.zne.inference import RichardsonFactory

# Change these according to your OpenCL devices and system specifications:
os.environ['QRACK_MAX_PAGE_QB']='27'
os.environ['QRACK_MAX_ALLOC_MB']='15872'
os.environ['QRACK_MAX_PAGING_QB']='30'
os.environ['QRACK_MAX_CPU_QB']='-1'
# os.environ['QRACK_QUNITMULTI_DEVICES']='1'
# os.environ['QRACK_QPAGER_DEVICES']='1'

In [3]:
def cx(circ, q1, q2):
    circ.cx(q1, q2)

def cy(circ, q1, q2):
    circ.cy(q1, q2)

def cz(circ, q1, q2):
    circ.cz(q1, q2)

def acx(circ, q1, q2):
    circ.x(q1)
    circ.cx(q1, q2)
    circ.x(q1)

def acy(circ, q1, q2):
    circ.x(q1)
    circ.cy(q1, q2)

def acz(circ, q1, q2):
    circ.x(q1)
    circ.cz(q1, q2)
    circ.x(q1)

def swap(circ, q1, q2):
    circ.swap(q1, q2)

def nswap(circ, q1, q2):
    circ.cz(q1, q2)
    circ.swap(q1, q2)
    circ.cz(q1, q2)

def pswap(circ, q1, q2):
    circ.cz(q1, q2)
    circ.swap(q1, q2)   

def mswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.cz(q1, q2)

def iswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.cz(q1, q2)
    circ.s(q1)
    circ.s(q2)

def iiswap(circ, q1, q2):
    circ.sdg(q2)
    circ.sdg(q1)
    circ.cz(q1, q2)
    circ.swap(q1, q2)

def random_circuit(width, depth, magic):
    circuit = QuantumCircuit(width)
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))
    two_bit_gates = swap, pswap, mswap, nswap, iswap, iiswap, cx, cy, cz, acx, acy, acz

    for i in range(depth):
        # Single bit gates
        for j in range(width):
            for _ in range(3):
                # We're trying to cover 3 Pauli axes
                # with Euler angle axes x-z-x. 
                circuit.h(j)

                # We can trace out a quarter rotations around the Bloch sphere with stabilizer.
                rnd = random.randint(0, 3)
                if rnd & 1:
                    circuit.s(j)
                if rnd & 2:
                    circuit.z(j)

                # For each axis, there is a chance of "magic."
                if (magic > 0) and ((3 * width * depth * random.random() / max_magic) < 1):
                    angle = random.uniform(0, math.pi / 2)
                    circuit.rz(angle, j)
                    magic -= 1

        # Nearest-neighbor couplers:
        ############################
        gate = gateSequence.pop(0)
        gateSequence.append(gate)
        for row in range(1, row_len, 2):
            for col in range(row_len):
                temp_row = row
                temp_col = col
                temp_row = temp_row + (1 if (gate & 2) else -1);
                temp_col = temp_col + (1 if (gate & 1) else 0)

                if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
                    continue

                b1 = row * row_len + col
                b2 = temp_row * row_len + temp_col

                if (b1 >= width) or (b2 >= width):
                    continue

                g = random.choice(two_bit_gates)
                g(circuit, b1, b2)
    
    return circuit

Run this next cell several times to produce a noisy value, (optionally tuning the width, depth, and magic settings above,) and run the last cell to mitigate with Mitiq.

In [4]:
def expit(x):
    if x == float("inf"):
        return 1.0
    if math.isnan(x) or (x == float("-inf")):
        return 0.0
    return 1 / (1 + math.exp(-x))

def execute(circuit):
    """Returns the mirror circuit expectation value for unsigned integer overall bit string."""

    circ = circuit.compose(circuit.inverse())
    qsim = QrackSimulator(circ.width(), isSchmidtDecompose=False, isOpenCL=False)
    qsim.run_qiskit_circuit(circ, 0)

    qubits = list(range(width))
    perm = [False] * width

    # This is fidelity of the mirror circuit:
    p = (qsim.prob_perm_rdm(qubits, perm, r=True) + qsim.prob_perm_rdm(qubits, perm, r=False)) / 2
    # So as not to exceed floor at 0.0 and ceiling at 1.0, (assuming 0 < p < 1,)
    # we mitigate its logit function value (https://en.wikipedia.org/wiki/Logit)
    return math.log(p / (1 - p)) 

circ = random_circuit(width, depth, max_magic)
logit_fidelity = execute(circ)
fidelity = expit(logit_fidelity)
print("Raw fidelity: ", fidelity)

Raw fidelity:  0.06347656249999999


In [5]:
if (1 - fidelity) > 1e-6:
    scale_count = 6
    max_scale = 2.5
    factory = RichardsonFactory(scale_factors=[(1 + (max_scale - 1) * x / scale_count) for x in range(0, scale_count)])
    print("Fidelity with ZNE: ", expit(zne.execute_with_zne(circ, execute, scale_noise=fold_global, factory=factory)))
else:
    print("No mitigation required. (Ideal value.)")

Fidelity with ZNE:  0.9920697210563717
